In [1]:
!pip install -q torch torchaudio transformers ffmpeg-python pydub pyannote.audio fastapi uvicorn python-multipart websockets
!sudo apt-get install -y ffmpeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:

In [2]:
from fastapi import FastAPI, WebSocket
from fastapi.middleware.cors import CORSMiddleware
import uvicorn
import asyncio

app = FastAPI()

# Allow CORS for WebSocket
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.websocket("/ws")
async def audio_stream(websocket: WebSocket):
    await websocket.accept()
    try:
        while True:
            audio_data = await websocket.receive_bytes()
            # Process audio chunk here (we'll add this later)
            print(f"Received {len(audio_data)} bytes")
    except Exception as e:
        print(f"Error: {e}")

def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Run in background
import threading
thread = threading.Thread(target=run_server)
thread.start()

In [3]:
from pydub import AudioSegment
import io

def convert_to_wav(audio_bytes, sample_rate=16000):
    """Convert raw audio bytes to WAV format"""
    audio = AudioSegment.from_file(io.BytesIO(audio_bytes))
    audio = audio.set_frame_rate(sample_rate).set_channels(1)
    wav_buffer = io.BytesIO()
    audio.export(wav_buffer, format="wav")
    return wav_buffer.getvalue()

In [5]:
!pip install whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for whisper: filename=whisper-1.1.10-py3-none-any.whl size=41120 sha256=ed2c28567b6c8425891f63de255c127a487653ff72d7d702460e7f0b65e87a84
  Stored in directory: /root/.cache/pip/wheels/21/65/ee/4e6672aabfa486d3341a39a04f8f87c77e5156149299b5a7d0
Successfully built whisper


In [11]:
!pip install -q openai-whisper pydub
!apt-get install -qq ffmpeg

import whisper
from pydub import AudioSegment
import os
from urllib.request import urlretrieve
from io import BytesIO

# Load Whisper model
model = whisper.load_model("base")

def convert_to_wav(mp3_path, wav_path):
    """Convert MP3 to WAV format"""
    try:
        audio = AudioSegment.from_mp3(mp3_path)
        audio.export(wav_path, format="wav")
        return True
    except Exception as e:
        print(f"Conversion failed: {str(e)}")
        return False

def transcribe_audio(audio_path):
    """Transcribe an audio file using Whisper"""
    try:
        result = model.transcribe(audio_path)
        return result["text"]
    except Exception as e:
        print(f"Transcription failed: {str(e)}")
        return None

# Download sample MP3
mp3_url = "https://www.soundhelix.com/examples/mp3/SoundHelix-Song-1.mp3"
mp3_filename = "sample.mp3"
wav_filename = "converted.wav"

try:
    print("Downloading sample audio...")
    urlretrieve(mp3_url, mp3_filename)

    if os.path.exists(mp3_filename):
        print("Converting to WAV...")
        if convert_to_wav(mp3_filename, wav_filename):
            print("Transcribing audio...")
            transcription = transcribe_audio(wav_filename)

            if transcription:
                print("\nTranscription Result:")
                print(transcription)
            else:
                print("Transcription returned empty result.")
    else:
        print("Failed to download audio file.")
except Exception as e:
    print(f"Error: {str(e)}")

Converting to WAV...
Transcribing audio...


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



Transcription Result:
 better change. I'm going to be doing this. I'm going to be doing this. I'm going to be doing this. I'm going to be doing this. I'm going to be doing this. I'm going to be doing this. I'm going to be doing this. I'm going to be doing this. I'm going to be doing this. I'm going to be doing this. I'm going to be doing this. I'm going to be doing this. I'm going to be doing this. I'm going to be doing this. I'm going to be doing this. I'm going to be doing this. I'm going to be doing this. I'm going to be doing this. I'm going to be doing this. I'm going to be doing this. I'm going to be doing this. I'm going to be doing this. I'm going to be doing this. I'm going to be doing this. I'm going to be doing this. I'm going to be doing this. I'm going to be doing this. I'm going to be doing this. I'm going to be doing this. I'm going to be doing this. I'm going to be doing this. I'm going to be doing this. I'm going to be doing this. I'm going to be doing this. I'm going

In [9]:
!pip install -q openai-whisper pydub
!apt-get install -qq ffmpeg

import whisper
from pydub import AudioSegment
import os
from urllib.request import urlretrieve
from io import BytesIO

# Load Whisper model
model = whisper.load_model("base")

def convert_to_wav(mp3_path, wav_path):
    """Convert MP3 to WAV format"""
    try:
        audio = AudioSegment.from_mp3(mp3_path)
        audio.export(wav_path, format="wav")
        return True
    except Exception as e:
        print(f"Conversion failed: {str(e)}")
        return False

def transcribe_audio(audio_path):
    """Transcribe an audio file using Whisper"""
    try:
        result = model.transcribe(audio_path)
        return result["text"]
    except Exception as e:
        print(f"Transcription failed: {str(e)}")
        return None

# Download sample MP3
mp3_url = "https://drive.google.com/file/d/10u1po6apL3H6aW5Y2dU4y18r2k_62Pmy/view?usp=sharing"
mp3_filename = "/content/Catching Up With Friends Audio 2.mp3"
wav_filename = "converted.wav"

try:
    print("Downloading sample audio...")
    urlretrieve(mp3_url, mp3_filename)

    if os.path.exists(mp3_filename):
        print("Converting to WAV...")
        if convert_to_wav(mp3_filename, wav_filename):
            print("Transcribing audio...")
            transcription = transcribe_audio(wav_filename)

            if transcription:
                print("\nTranscription Result:")
                print(transcription)
            else:
                print("Transcription returned empty result.")
    else:
        print("Failed to download audio file.")
except Exception as e:
    print(f"Error: {str(e)}")

Converting to WAV...
Conversion failed: Decoding failed. ffmpeg returned error code: 1

Output from ffmpeg/avlib:

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt -

In [10]:
# Install required packages
!pip install -q openai-whisper pydub
!apt-get install -qq ffmpeg

import whisper
from pydub import AudioSegment
import os
import requests
from io import BytesIO

# Load Whisper model (using 'tiny' for faster results in Colab)
model = whisper.load_model("tiny")

def download_audio(url, filename):
    """Download audio file with error handling"""
    try:
        response = requests.get(url)
        response.raise_for_status()
        with open(filename, 'wb') as f:
            f.write(response.content)
        return True
    except Exception as e:
        print(f"Download failed: {str(e)}")
        return False

def convert_to_wav(input_path, output_path):
    """Convert audio to WAV format with robust error handling"""
    try:
        # Try multiple audio formats
        for fmt in ['mp3', 'wav', 'ogg']:
            try:
                audio = AudioSegment.from_file(input_path, format=fmt)
                audio.export(output_path, format="wav")
                return True
            except:
                continue
        print("Failed to convert: Unsupported audio format")
        return False
    except Exception as e:
        print(f"Conversion error: {str(e)}")
        return False

def transcribe_audio(audio_path):
    """Transcribe audio with error handling"""
    try:
        result = model.transcribe(audio_path)
        return result["text"]
    except Exception as e:
        print(f"Transcription failed: {str(e)}")
        return None

# Using a reliable test audio URL (short English speech)
audio_url = "https://www2.cs.uic.edu/~i101/SoundFiles/gettysburg10.wav"
input_filename = "input_audio"
output_filename = "output.wav"

# Download and process
if download_audio(audio_url, input_filename):
    if convert_to_wav(input_filename, output_filename):
        transcription = transcribe_audio(output_filename)
        if transcription:
            print("\nSuccessful Transcription:")
            print(transcription)
        else:
            print("Transcription failed")
    else:
        print("Conversion failed")
else:
    print("Audio download failed")

# Fallback: Use Whisper's built-in audio loading if all else fails
if not os.path.exists(output_filename):
    print("\nTrying direct Whisper processing...")
    try:
        transcription = transcribe_audio(input_filename)
        if transcription:
            print("\nFallback Transcription:")
            print(transcription)
    except:
        print("All methods failed - please try a different audio file")

100%|█████████████████████████████████████| 72.1M/72.1M [00:01<00:00, 60.9MiB/s]


Download failed: HTTPSConnectionPool(host='www2.cs.uic.edu', port=443): Max retries exceeded with url: /~i101/SoundFiles/gettysburg10.wav (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7e316a9ac6d0>, 'Connection to www2.cs.uic.edu timed out. (connect timeout=None)'))
Audio download failed

Trying direct Whisper processing...


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcription failed: Failed to load audio: ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidst